In [1]:
# 데이터 불러오기
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AICon with STOLIO/drama recommendation/model_dev/data.csv',engine='python',encoding='CP949')
data.tail()

,index,name,synopsis,seezn,series_on,watcha,wavve,netflix,tving
308,308,SKY 캐슬,"대한민국 상위 0.1%가 모여 사는 SKY 캐슬 안에서 남편은 왕으로, 제 자식은 ...",1,1,0,0,1,1
309,309,어비스,′′영혼 소생 구슬′′ 어비스를 통해 생전과 180도 다른 ′반전 비주얼′로 부활한...,1,1,0,0,1,1
310,310,손 the guest,"한국 사회 곳곳에서 기이한 힘에 의해 벌어지는 범죄에 맞서는 영매와 사제, 형사의 ...",1,1,0,0,1,1
311,311,열여덟의 순간,위태롭고 미숙한 ′Pre-청춘′들의 세상을 있는 그대로 들여다보는 감성 청춘 드라마,1,1,0,0,1,1
312,312,남자친구,한 번도 자신이 선택한 삶을 살아보지 못한 수현과 자유롭고 맑은 영혼 진혁의 우연한...,1,1,0,0,1,1


In [2]:
data.shape

(313, 9)

In [3]:
synopsis = data['synopsis']
print(synopsis[:5])

0    천재적인 두뇌의 소유자 우영우. 대형 로펌의 신입 변호사이자 자폐 스펙트럼 장애를 ...
1    위장 잠입 수사를 하다가 교도소에 수감된 사법연수생. 교도소에서 살아남아 복수하기 ...
2    타협은 없다. 용서도 없다. 썩은 놈들이 판치는 대한민국, 그놈들 잡으러 나선 나쁜...
3    갑작스러운 테러로 대참사를 맞은 대한민국. 대통령, 국무총리, 국회의장, 그 모두를...
4    통일 한국의 조폐국을 강도단이 장악했다. 인질들이 건물 안에 갇혀 있는 상황. 경찰...
Name: synopsis, dtype: object


# 데이터를 전처리 한다.
- 한국어 조사처리를 이용할 예정

In [4]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 54.8 MB/s 


In [5]:
from konlpy.tag import Okt

okt = Okt()
synopsis_array = []
for i in data['index']:
  synopsis_array.append(" ".join(okt.nouns(synopsis[i])))

In [6]:
data['synopsis_preprocessing'] = synopsis_array

In [7]:
data.head()

,index,name,synopsis,seezn,series_on,watcha,wavve,netflix,tving,synopsis_preprocessing
0,0,이상한 변호사 우영우,천재적인 두뇌의 소유자 우영우. 대형 로펌의 신입 변호사이자 자폐 스펙트럼 장애를 ...,1,0,0,0,1,0,천재 두뇌 소유자 우영 대형 로펌 신입 변호사 이자 자폐 스펙트럼 장애 여성 법정 ...
1,1,인사이더,위장 잠입 수사를 하다가 교도소에 수감된 사법연수생. 교도소에서 살아남아 복수하기 ...,1,1,0,0,1,0,위장 잠입 수사 교도소 수감 사법 수생 교도소 복수 위해 재소 자 불법 도박 판
2,2,나쁜녀석들: 악의 도시,"타협은 없다. 용서도 없다. 썩은 놈들이 판치는 대한민국, 그놈들 잡으러 나선 나쁜...",1,1,0,0,1,0,타협 용서 놈 대한민국 놈 나선 녀석 방 검사 판 조폭 형사 발 전쟁 시작
3,3,"60일, 지정생존자","갑작스러운 테러로 대참사를 맞은 대한민국. 대통령, 국무총리, 국회의장, 그 모두를...",1,1,0,0,1,0,테러 참사 대한민국 대통령 국무총리 국회의장 그 모두 혼란 속 환경부 장관 박무진 ...
4,4,종이의 집 공동경제구역,통일 한국의 조폐국을 강도단이 장악했다. 인질들이 건물 안에 갇혀 있는 상황. 경찰...,0,0,0,0,1,0,통일 한국 폐국 강도 단 장악 인질 건물 안 상황 경찰 게 강도 제압 이 작전 설계...


In [8]:
# 데이터 벡터화 진행
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(data['synopsis_preprocessing'])
print(tfidf_matrix.shape)

(313, 2275)


In [9]:
# 코사인 유사도 과정 진행
from sklearn.metrics.pairwise import linear_kernel

cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cos_sim.shape)

(313, 313)


In [10]:
# 영화제목을 입력하면 인덱스를 가져온다.
indices = pd.Series(data.index, index=data['name']).drop_duplicates()
indices

name
이상한 변호사 우영우       0
인사이더              1
나쁜녀석들: 악의 도시      2
60일, 지정생존자        3
종이의 집 공동경제구역      4
               ... 
SKY 캐슬          308
어비스             309
손 the guest     310
열여덟의 순간         311
남자친구            312
Length: 313, dtype: int64

In [11]:
indices['인사이더']

1

In [12]:
# 영화의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 영화 목록 반환
import numpy as np

def get_recommendations(title, cosine_sim=cos_sim):
  # return data type
  return_dict = {}
  ott_dict = {}

  # ott 가격정보 상수화하기
  seezn_price = 6300
  watcha_price = 7900
  wavve_price =7900
  netflix_price = 9500
  tiving_price = 7900  

  idx = indices[title] # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기

  # 코사인 유사도 매트릭스 에서 idx에 해당하는 데이터를 (idx, 유사도 형태로 얻기)
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 정렬하기
  # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
  sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)

  # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
  sim_scores = sim_scores[1:11] # 자기 자신을 제외함

  # 추천 영화 목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  for item in movie_indices:
    return_dict[data['name'].iloc[item]] = list(data.iloc[item, 3:-1])

  # print(return_dict)
  # 가성비 확인 ott dict 만들기
  ott_list = []
  for _, ott in return_dict.items():
    ott_list.append(ott)
  ott_nparray = np.array(ott_list)
  # print(ott_list)
  ott_sum = np.sum(ott_nparray, axis=0)
  # print(ott_sum)

  try :
    ott_dict['seezn'] = int(seezn_price / ott_sum[0])
  except:
    ott_dict['seezn'] = seezn_price
  # seris_on은 편당 가격제이기 때문에 가성비 체크에서 제외
  try:
    ott_dict['watcha'] = int(watcha_price / ott_sum[2])
  except:
    ott_dict['watcha'] = watcha_price
  try:
    ott_dict['wavve'] = int(wavve_price / ott_sum[3])
  except:
    ott_dict['wavve'] = wavve_price
  try:
    ott_dict['netflix'] = int(netflix_price / ott_sum[4])
  except:
    ott_dict['netflix'] = netflix_price
  try:
    ott_dict['tiving'] = int(tiving_price / ott_sum[5])
  except:
    ott_dict['tiving'] = tiving_price
  # print(ott_dict)

  # 최선의 선택 출력
  vest_choice = np.argmin(np.array(ott_dict.values))
  # print(vest_choice)
  ott_dict['vest_choice'] = 0 if vest_choice == 0 else vest_choice + 1

  # 인덱스 값을 통해서 영화 제목을 얻어오기(return)
  return return_dict, ott_dict

In [13]:
get_recommendations("빈센조")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in long_scalars


({'리갈하이': [1, 1, 0, 0, 0, 1],
  '손 the guest': [1, 1, 0, 0, 1, 1],
  '우아한 가': [1, 1, 0, 1, 0, 0],
  '하이에나': [1, 1, 0, 1, 1, 0],
  '무법변호사': [1, 1, 0, 0, 1, 0],
  '이상한 변호사 우영우': [1, 0, 0, 0, 1, 0],
  '내일 지구가 망해버렸으면 좋겠어': [0, 0, 0, 0, 1, 0],
  '종이의 집 공동경제구역': [0, 0, 0, 0, 1, 0],
  '경여년': [1, 1, 0, 0, 1, 1],
  '바벨': [1, 1, 0, 1, 0, 0]},
 {'seezn': 787,
  'watcha': 7900,
  'wavve': 2633,
  'netflix': 1357,
  'tiving': 2633,
  'vest_choice': 0})

In [14]:
ott_idx = {
    0 : "seezn",
    1 : "series_on",
    2 : "watcha",
    3 : "wavve",
    4 : "netflix",
    5 : "tiving"
}

In [15]:
import pickle

movies = data[['index', 'name', 'synopsis', 'seezn', 'series_on', 'watcha', 'wavve', 'netflix', 'tving'	]]

pickle.dump(movies, open('movies.pickle', 'wb'))
pickle.dump(cos_sim, open('cosine_sim.pickle', 'wb'))